# Model Geometry Construction using Python 

As part of CE 5304 - Advanced Design of Structural Systems - at The University of Texas at El Paso, I have encouraged my students to think about constructing structural model geometry programmatically. The purpose of this Notebook is to help support that initiative with an interactive example.  

## Model Construction Considerations and Commonalities 

Every structural model is built on several key aspects, which are software agnostic. These include:
+ Geometry
+ Material Properties
+ Section Properties
+ Connectivity
+ Boundary Conditions
+ Loads

While this list is not exhaustive, these core components are critical to every model build, and are generally the drivers behind modeling errors. Currently we are just looking at the top three. 

## Why build geometry programmatically?

There are numerous reasons to automate as much of your model construction as possible. They include:
+ It is faster
+ You are far less likely to make errors
+ It is much easier to make changes
+ It creates a logical backbone to your model construction that can be leveraged throughout the process

Before we start we need to initialize a few packages that we will use. 

In [40]:
# Add in imports as required
import math
import pprint
pp = pprint.PrettyPrinter(indent=4)
import csv
import html

from IPython.display import HTML

import base64  
import pandas as pd  


## Our Example Building 

Consider the following building specifications:
+ Three stories with a story height of 18 feet
+ Outer dimensions of 120' by 120' 
+ Bay size of 30' by 30' 
+ Two intermediate beams running transverse in each bay, spaced at 10'

### Explore the building dimensions

We need to make sure we understand the layout and dimensions of the proposed building. Let's define the given values above. Note that we are ignoring many, many components of the building that may complicate this process. 

In [29]:
# Input Variables
storyHeight=18 #feet
numStories=4 #this equates to four levels of nodes but three floors - G/1/2/Roof
bldgHeight=storyHeight*(numStories-1) #building height includes three 18' stories
x_length=120 #feet
y_length=120 #feet
baySizeX=30 #feet
baySizeY=30 #feet
beamSpacingX=30 #feet
beamSpacingY=10 #feet

### Generating our geometry
My goal with this next section is create a flexible geometry generation approach that can be used for other building structures. In other words, the process is repeatable and reliable, and it is not a one-off effort that would have to be repeated for different rectangular structures. 

#### Nodal Geometry
The nodal geometry will be generated first. We will use three nested for loops. 
+ Variable i corresponds to the number of stories in the building. 
+ Variable j corresponds to the number of beams in the X-direction.
+ Variable k corresponds to the number of beams in the Y-direction.

Note that as of now, we are not making use of the "baySize" variables. They will be used for columns generation.

Node numbers are assigned in groups of 1000, meaning that floor 0 will have numbers in the 1000-1999 range, and floor 1 will have numbers in the 2000-2999 range and so on. We are generating nodes on the base floor that will not have members associated with them. These can be ignored or manually removed later. Also note that we are numbering non-consecutively, meaning that if a software requires consecutive numbering, it may have to be redone later, or a different approach should be adopted. 

In [30]:
# Generate the nodes 
NODES={}
for i in range(1,math.ceil(numStories+1)):
    for j in range(1,math.ceil(x_length/beamSpacingX)+2):
        for k in range(1,math.ceil(y_length/beamSpacingY)+2):
            n=(1000*i)+(j-1)*(y_length/beamSpacingY+1)+k
            x=(j-1)*beamSpacingX
            y=(k-1)*beamSpacingY
            z=(i-1)*storyHeight
            NODES[n]=[x,y,z]
        


#### Reviewing results
The following command will allow you to view your nodal results if interested. If not, leave it commented. 

In [31]:
# print the Nodes generated to take a look at them 
#pp.pprint(NODES)

### Beam Element Designation
The Following will generate the beam element input. It is approached in two sets of nested for loops. The first will generate the beams that move in the y-direction. The second will generate the beams in the x-direction.

Important Note: This code assumes that beams within bays are in the x-direction. If this is not the case, then adjustments should be made. 

The format for the data being written into the BEAMS dictionary follows the requirements for SkyCiv and would have to be modified for different software. 

Again, non-consecutive element numbering is used. 

In [32]:
# generate the beam elements 
BEAMS={}
for i in range(1,math.ceil(numStories)):
    for j in range(1,math.ceil(x_length/beamSpacingX)+2): 
        for k in range(1,math.ceil(y_length/beamSpacingY)+1): 
            mID=(10000*(i+1))+(1000*j)+k
            nodeA=(1000*(i+1))+k+(j-1)*(y_length/beamSpacingY+1)
            nodeB=nodeA+1
            typ=1
            sectID=1
            rot=0
            fixA='FFFFFR'
            fixB='FFFFFR'
            offAX=0
            offAY=0
            offAZ=0
            offBX=0
            offBY=0
            offBZ=0
            cbl=0
            tc=0
            BEAMS[mID]=[nodeA,nodeB,typ,sectID,rot,fixA,fixB,offAX,offAY,offAZ,offBX,offBY,offBZ,cbl,tc]
            
for i in range(1,math.ceil(numStories)):
    for j in range(1,math.ceil(x_length/beamSpacingX)+1): 
        for k in range(1,math.ceil(y_length/beamSpacingY)+2): 
            mID=(10000*(i+1))+(100*j)+k
            nodeA=(1000*(i+1))+k+(j-1)*(y_length/beamSpacingY+1)
            nodeB=nodeA+(y_length/beamSpacingY+1)
            typ=1
            sectID=1
            rot=0
            fixA='FFFFFR'
            fixB='FFFFFR'
            offAX=0
            offAY=0
            offAZ=0
            offBX=0
            offBY=0
            offBZ=0
            cbl=0
            tc=0
            BEAMS[mID]=[nodeA,nodeB,typ,sectID,rot,fixA,fixB,offAX,offAY,offAZ,offBX,offBY,offBZ,cbl,tc]           

#### Reviewing results
The following command will allow you to view your beam results if interested. If not, leave it commented. 

In [33]:
#pp.pprint(BEAMS)

### Column Element Designation
The Following will generate the column element input. It is approached in a set of nested for loops. It makes use of the baySize variables to ensure that columns are created only at the corner of bays. 

The format for the data being written into the columns dictionary follows the requirements for SkyCiv and would have to be modified for different software. 

Again, non-consecutive element numbering is used. 

In [34]:
# generate the column elements 
COLUMNS={}
for i in range(1,math.ceil(numStories)):
    for j in range(1,math.ceil(x_length/baySizeX)+2): 
        for k in range(1,math.ceil(y_length/baySizeY)+2): 
            mID=(100000*(i))+(1000*j)+k
            nodeA=(1000*i)+(1+(k-1)*(baySizeY/beamSpacingY))+(j-1)*(y_length/beamSpacingY+1)
            nodeB=nodeA+1000
            typ=1
            sectID=1
            rot=0
            fixA='FFFFFF'
            fixB='FFFFFF'
            offAX=0
            offAY=0
            offAZ=0
            offBX=0
            offBY=0
            offBZ=0
            cbl=0
            tc=0
            COLUMNS[mID]=[nodeA,nodeB,typ,sectID,rot,fixA,fixB,offAX,offAY,offAZ,offBX,offBY,offBZ,cbl,tc]

#### Reviewing results
The following command will allow you to view your column results if interested. If not, leave it commented. 

In [35]:
#pp.pprint(COLUMNS)

In [42]:
# convert dictionary to data frame in pandas
df_nodes=pd.DataFrame.from_dict(NODES, orient='index')

In [43]:
# create an HTML link to download your generated geometries

def create_download_link( df_nodes, title = "Download CSV file", filename = "data.csv"):  
    csv = df_nodes.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_nodes)

In [44]:
# convert dictionary to data frame in pandas
df_beams=pd.DataFrame.from_dict(BEAMS, orient='index')

In [45]:
# create an HTML link to download your generated geometries

def create_download_link( df_beams, title = "Download CSV file", filename = "data.csv"):  
    csv = df_beams.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_beams)

In [46]:
# convert dictionary to data frame in pandas
df_columns=pd.DataFrame.from_dict(COLUMNS, orient='index')

In [47]:
# create an HTML link to download your generated geometries

def create_download_link( df_columns, title = "Download CSV file", filename = "data.csv"):  
    csv = df_columns.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_columns)